In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Multiply
from tensorflow.keras.activations import relu
from DataLoader import DataLoader
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model

import numpy as np

In [2]:
training_generator = DataLoader(64, "train", shuffle=True)
val_generator = DataLoader(64, "val")

In [3]:
lossHistory = []
valHistory = []
class NnomCallbacks(tf.keras.callbacks.Callback):
  def on_batch_end(self, batch, logs=None):
    lossHistory.append(logs['loss'])
    if batch and batch % 1000 == 0:
      # pred = self.model.predict(val_generator)
      # loss = (pred.flatten() - val_generator.labels.flatten()[0:len(val_generator)*64])**2
      # loss = np.sum(loss) / len(loss)
      # valHistory.append(loss)

      # plt.plot(valHistory)
      # plt.savefig("val.png", dpi=200)
      # plt.clf()

      plt.plot(lossHistory[20:])
      plt.savefig("loss.png", dpi=200)
      plt.clf()
      
      # print(" val_loss : {}".format(loss))

      # self.model.save("models/model-{}-{}".format(len(valHistory), int(loss*10000)))

In [ ]:
i_main = Input((98304,), sparse=True)
i_legal = Input((384,), sparse=True)

layerStack = Dense(512, activation='relu')(i_main)
layerStack = Dense(384, activation='sigmoid')(layerStack)
layerStack = Multiply()([layerStack, i_legal])

model = Model(inputs=[i_main, i_legal], outputs=layerStack)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model = tf.keras.models.load_model("production/")

In [5]:
model.fit(training_generator, validation_data=val_generator, epochs=1, callbacks=[NnomCallbacks()])

 64250/184471 [=========>....................] - ETA: 1:04:46 - loss: 2.3745 - accuracy: 0.2926

KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [1]:
model.save("production/")

NameError: name 'model' is not defined

In [ ]:
import time
model.save("models/model-{}".format(int(time.time())))